**Beam-ის დაყენება; საჭირო იმპორტები**

In [1]:
%pip install --quiet apache-beam

from enum import Enum

import apache_beam as beam
import apache_beam as beam

from apache_beam.dataframe import convert
from apache_beam.dataframe.io import read_csv, to_csv

import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

import re
from collections import Counter

# თუ ეს error-ები და warning-ები გექნებათ, შეგიძლიათ დააიგნოროთ: 
# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.
# ან
# WARNING:apache_beam.runners.interactive.interactive_environment:Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.
# WARNING:root:Make sure that locally built Python SDK docker image has Python 3.7 interpreter.


     |████████████████████████████████| 9.8 MB 5.2 MB/s 
     |████████████████████████████████| 62 kB 842 kB/s 
     |████████████████████████████████| 151 kB 57.7 MB/s 
     |████████████████████████████████| 249 kB 49.9 MB/s 
     |████████████████████████████████| 2.3 MB 36.3 MB/s 
     |████████████████████████████████| 829 kB 37.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
pip show apache_beam

Name: apache-beam
Version: 2.34.0
Summary: Apache Beam SDK for Python
Home-page: https://beam.apache.org
Author: Apache Software Foundation
Author-email: dev@beam.apache.org
License: Apache License, Version 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: hdfs, pyarrow, pydot, grpcio, oauth2client, httplib2, future, typing-extensions, dill, pymongo, crcmod, protobuf, pytz, requests, orjson, fastavro, avro-python3, numpy, python-dateutil
Required-by: 


# **ამოცანები**

**ამოცანა #1 - სტუდენტების საბოლოო ნიშნების დათვლა**
*(1 ქულა)*

ამ ამოცანის მიზანია, რამოდენიმე სტუდენტისთვის ერთ-ერთი საგნის ფინალური ქულა დავთვალით და გავიგოთ, ჩააბარა თუ არა სტუდენტმა ეს საგანი. კრიტერიუმები მარტივია და სულ ორი კომპონენტია:

 1) შუალედური გამოცდა, რომლის ქულებიც თითოეული სტუდენტისთვის მოცემულია midterm_grades ცვლადში. ეს არის tuple-ების მასივი. tuple-ის პირველი ელემენტია სტუდენტის სახელი, მეორე - გამოცდაში მიღებული ქულა;
 
 2) ფინალური პროექტი, რომლის ქულებიც თითოეული სტუდენტისთვის მოცემულია project_grades ცვლადში. ესეც არის tuple-ების მასივი. 

საგნის ჩააბარების კრიტერიუმი: თუ სტუდენტის საშუალო ქულა 50.0-ია, მაშინ ვთვლით, რომ სტუდენტმა საგანი ჩააბარა.

ამოცანაში უნდა დაითვალოთ სტუდენტების საშუალო ქულა ორივე კომპონენტში და საბოლოო შედეგი წარმოადგინოთ pcollection-ის სახით. ამ კოლექციის თითოეული ელემენტი უნდა იყოს დაფორმატებული ასეთნაირად:

*Student1 has passed the subject with avg grade of 100.0*




---



ამოცანა უნდა შეასრულოთ Beam-ის გამოყენებით. ამისთვის გაწვდით მინიმალისტურ ამოხსნას (უფრო სტრუქტურას), რომელიც თქვენ უნდა გაარჩიოთ და დაასრულოთ. გაითვალისწინეთ:
 * ამოხსნის ძირითადი ჩონჩხი უკვე აწყობილია. თქვენ საწყისი pcollection-ები სწორად უნდა დააგენერიროთ და დაასრულოთ apply_transforms ფუნქციის იმპლემენტაცია. apply_transforms არის ის ადგილი, სადაც ყველა მოთხოვნილი ტრანფორმაცია ხდება pcollection-ებზე.
 * ფორმატირებისთვის უნდა დაიხმაროთ კლასი StudentGrade. რა თქმა უნდა, მისი იმპლემენტაციაც თქვენი მოსაფიქრებელია. 🐱‍💻

In [3]:
midterm_grades = [('Student1', 100.0),('Student2', 50.0),('Student3', 75.0)]
project_grades = [('Student1', 100.0),('Student2', 50.0),('Student4', 90.0),('Student5', 99.0)]

PASSING_GRADE = 50.0

class StudentGrade:
    
  
    def __init__(self, student_name, midterm_grade, project_grade):
        self.student_name = student_name
        self.avg_grade = float(((midterm_grade or 0) + (project_grade or 0))) / 2.0

    def __str__(self):
        grade_string = self.student_name + " has "
        grade_string += "passed" if self.avg_grade >= PASSING_GRADE else "failed"
        grade_string += " the subject with avg grade of " + str(self.avg_grade)
        return grade_string

def apply_transforms(midterm_grades, project_grades):

    def cogbk_result_to_studentgrade(cgbk_result):
        (name, grade) = cgbk_result
        if len(grade['midterm']) == 0: 
            grade['midterm'].append(0)
        if len(grade['project']) == 0: 
            grade['project'].append(0)   
        return StudentGrade(name, grade['midterm'][0], grade['project'][0])
 
    return ({'midterm': midterm_grades, 'project': project_grades}
            | beam.CoGroupByKey()
            | beam.Map(cogbk_result_to_studentgrade))
          

with beam.Pipeline() as p:
    midterm_collection = (p | 'Midterm' >> beam.Create(midterm_grades))
    project_collection = (p | 'Project' >> beam.Create(project_grades))

    (
      apply_transforms(midterm_collection, project_collection)
        | beam.Map(print)
    )

Student1 has passed the subject with avg grade of 100.0
Student2 has passed the subject with avg grade of 50.0
Student4 has failed the subject with avg grade of 45.0
Student5 has failed the subject with avg grade of 49.5
Student3 has failed the subject with avg grade of 37.5


**ამოცანა #2 - ისევ და ისევ Marvel and DC: the good and the bad**
*(2 ქულა)*


უნდა ვიპოვოთ Top 10 ყველაზე ძლევამოსილი პერსონაჟი Marvel-ისა და DC-ის სამყაროებიდან Beam-ის გამოყენებით.

Dataset არის თქვენთვის უკვე ნაცნობი (ავად თუ კარგად 😁). სულ გვაქვს ორი csv ფაილი:
* characters_info.csv - პერსონაჟების შესახებ მონაცემები (სახელი, კეთილია/ბოროტია, სქესი, ფიზიკური აღწერილობა, რომელი სამყაროს ნაწილია და ა. შ. ველები: Name, Alignment, Gender, EyeColor, Race, HairColor,	Publisher,	SkinColor,	Height, Weight;
* character_stats.csv - სხვადასხვა პერსონაჟის სტატისტიკური მონაცემები ველებით Name, Alignment, Intelligence, Strength, Speed, Durability, Power, Combat, Total (ჯამური ქულა ყველა კომპონენტში);

დუბლირებებზე შეგიძლიათ არ იდარდოთ. ასევე, რა პერსონაჟებიც არიან ერთ ფაილში, იგივე პერსონაჟებია მეორეშიც და პირიქით. ფაილებში პერსონაჟები არა მარტო მარველის ან dc-ის, არამედ სხვა სამყაროებიდანაცაა (Tolkien, Harry Potter...) .

რომ დავაკონკრეტოთ ამოცანა:
  * უნდა მოვძებნოთ Top 10 ყველაზე ძლევამოსილი პერსონაჟები მხოლოდ და მხოლოდ Marvel-ის ან DC-ის სამყაროებიდან. რაც უფრო მაღალია Total ველის მნიშვნელობა (stats ფაილი), მით უფრო ძლევამოსილია პერსონაჟი;
  * გვაინტერესებს Top 10 ყველაზე ძლევამოსილი პერსონაჟი როგორც კეთილ გმირებში (alignment = 'good'), ასევე ბოროტებში (alignment = 'bad') **ცალ-ცალკე**. ანუ ტოპ 10 ყველაზე ძლიერი, კეთილი პერსონაჟი და იგივე ბოროტებისთვისაც. ყველა სხვა alignment არ გვაინტერესებს;
  * საბოლოო შედეგში გვინდა, რომ გვქონდეს მხოლოდ შემდეგი ველები: name, score (იგივე, რაც Total ველი), gender, race, universe. ეს შედეგები თითოეული aligment-ისთვის უნდა შევინახოთ ცალ-ცალკე ფაილში: top_10_bad_heroes.csv და top_10_good_heroes.csv. ანუ სულ ეს ორი output ფაილი გვექნება. ათეული დალაგებული უნდა იყოს score-ის კლებადობის მიხედვით.


**ტექნიკური დეტალები:**
 
 ამოხსნა მთლიანად თქვენს ფანტაზიაზეა, მაგრამ გარკვეულ რჩევებს მაინც მოგცემთ:
 * ამ ამოცანას უფრო იოლად ამოხსნით, თუ dataframe-ებს გამოიყენებთ;
 * იმისთვის, რომ dataframe-ები ვიზუალურად ნახოთ, დაგჭირდებათ Interactive Beam და მისი Runner, რომელთა შესაბამისი იმპორტებიც უკვე უზრუნველყოფილია. ***ib.collect(df)***-ით შეძლებთ dataframe ნახოთ საჭიროების შემთხვევაში. Pipeline-ის დასაწყისი დატოვეთ ისე, როგორც მოგაწოდეთ;
 * Beam-ის DataFrame-ები Pandas-ისას სინტაქსურად ძალიან ჰგავს, ოღონდ გარკვეული შეზღუდვებიც აქვს. მაგალითად, ინდექსების გარეშე Beam-ს პრობლემები აქვს ზოგ სიტუაციაში (თუნდაც merge-ისას) და ამის გამო გარკვეული exception-ები შეიძლება გქონდეთ. ამიტომ ინდექსები გაითვალისწინეთ;
 * მნიშვნელოვანი საკითხი: Beam-ის Pipeline-ში არც ერთი ოპერაციის label არ უნდა მეორდებოდეს. თუ რაღაც მიზეზებით ერთი და იგივე ტრანსფორმაცია ორჯერ გაქვთ ფაიფლაინში და სახელი საერთოდ არ მიუთითეთ, default-ად ერთსა და იმავე სახელს მიანიჭებს Beam და გექნებათ exception. ამიტომ გაითვალისწინეთ, რომ სხვადასხვა label უნდა მიუწეროთ ყველა ტრანსფორმაციას. Label-ები ორნაირად შეიძლება მიეთითოს, სიტუაციიდან გამომდინარე. მაგალითად: **pipeline | 'ლეიბლის უნიკალური სახელი' >> ტრანსფორმაცია**  ან **beam.Map(..., label='უნიკალური სახელი')**;
 * class Alignment(Enum): უბრალოდ enum-ია, სადაც alignment-ები გვაქვს გადანომრილი. აუცილებელი არაა გამოყენება, სურვილისამებრ;
 * Top 10 პერსონაჟის საპოვნელად, სავარაუდოდ, დაგჭირდებათ beam.transforms.combiners მოდულის ფუნქციები. თუ სხვა იდეა არ გექნებათ, მაგალითად, შეგიძლიათ ნახოთ აი, ეს ფუნქცია: [Top.Of](https://beam.apache.org/releases/pydoc/2.2.0/apache_beam.transforms.combiners.html#apache_beam.transforms.combiners.Top). [სხვა მაგალითებიც](http://beam.apache.org/documentation/transforms/python/aggregation/top/)
 * აუცილებელი არაა, მაგრამ ყოველი შემთხვევისთვის: თუ თქვენი solution-ის რაღაც მომენტში გექნებათ სიტუაცია, სადაც pipeline-ში გაქვთ pcollection სიის სახით და გინდათ, მიიღოთ სათითაოდ ყველა ელემენტი ამ სიაში, შეგიძლიათ, გამოიყენოთ ჩვენ მიერ მოწოდებული პაწაწინა კლასი BreakList;

 

In [4]:
class Alignment(Enum):
  bad = 0 
  good = 1
  neutral = 2
  undefined = 3

class BreakList(beam.DoFn):
  def process(self, element):
    for e in element:
      yield e


with beam.Pipeline(InteractiveRunner()) as pipeline:
    stats_df = pipeline | 'Read stats csv' >> beam.dataframe.io.read_csv('character_stats.csv')
    info_df = pipeline | 'Read info csv' >> beam.dataframe.io.read_csv('characters_info.csv')

    info_df = info_df.drop('Alignment', axis=1)
    data = stats_df.merge(
        info_df.set_index('Name'),
        right_index=True,
        left_on='Name',
        how='inner'
    )

    data = data[['Alignment','Total', 'Gender', 'Race', 'Publisher']]
    data = data.rename(columns = {
          "Total" : "score",
          "Gender" : "gender",
          "Race" : "race",
          "Publisher" : "universe"
        }
    )
    cond1 = data['universe'] == 'DC Comics'
    cond2 = data['universe'] == 'Marvel Comics'

    data = data[(cond1 | cond2)]
    data_good = data[data['Alignment'] == 'good'].drop("Alignment", axis=1)
    data_bad = data[data['Alignment'] == 'bad'].drop("Alignment", axis=1)
    data_good = data_good.nlargest(10, 'score', keep='any')
    data_bad = data_bad.nlargest(10, 'score', keep='any')
    data_good.to_csv('top_10_good_heroes.csv')
    data_bad.to_csv('top_10_bad_heroes.csv')

/usr/local/lib/python3.7/dist-packages/apache_beam/dataframe/io.py:572: FutureWarning: WriteToFiles is experimental.
  sink=lambda _: _WriteToPandasFileSink(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/fileio.py:550: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  p.options.view_as(GoogleCloudOptions).temp_location or


**ამოცანა #3 - Breaking News**
*(3 ქულა)*

დატასეტში bbc_news.csv მოცემული გვაქვს 1500-მდე მოკლე სტატია BBC News-იდან. თითოეულ სტატიას აქვს უნიკალური ID და კატეგორიის label-ი (tech, business, politics, entertainment, sport).

მიზანია, რომ თითოეულ სტატიას დავუგენერიროთ tag-ები შემდეგი მარტივი კრიტერიუმით:  უნდა მოვძებნოთ 3 ყველაზე ხშირი სიტყვა ამ სტატიაში. აუცილებელია, რომ თითოეული სიტყვა მინიმუმ 10-ჯერ მაინც გვხვდებოდეს კონკრეტულ სტატიაში.

გაითვალისწინეთ, რომ:
 * Tag-ები არ უნდა იყოს ე.წ. stopword-ები ან სიმბოლოები. მათი ჩამონათვალი არის ფაილში stopwords.txt;
 * Tag-ები არ შეიძლება იყოს რიცხვები;
 * Tag-ები არ შეიძლება იყოს ერთ ან ორ სიმბოლოიანი სიტყვები;
 * სიტყვებად ჩავთვალოთ შემდეგი სიმბოლოებით დაყოფილი ტოკენები: 

          whitespace

          ,

          :

          !

          ?

 * ტრანსფორმაციები უნდა შეასრულოთ, რასაკვირველია, Beam-ის გამოყენებით და არა უბრალოდ პითონის ხარჯზე. 😏

საბოლოო resultset უნდა ჩაწეროთ csv ფაილად და უნდა შედგებოდეს ველებისგან: ***ArticleID, Category, Tag, Frequency***


In [5]:
class ExplodeWords(beam.DoFn):
  def process(self, row):
    exploded = []
    word_counts = row['counts']
    for word_count in word_counts:
      exploded.append({"ArticleId" : row["ArticleId"],
                       "Category" :  row["Category"],
                       "Tag": word_count[0],
                       "Frequency": word_count[1]})
    return exploded


class FilterText(beam.DoFn):
  def process(self, row, stopwords):
    word_counts = row['Text']
    filtered_counts = Counter()
    for word, count in word_counts.items():
      if count >= 10 and len(word) > 2 and not word.isnumeric() and word not in stopwords:
        filtered_counts[word] = count
    row['counts'] = filtered_counts.most_common(3) 
    yield row


#replace text with split version
class SplitText(beam.DoFn):
    def process(self, row):
      text = row['Text']
      words = re.split('\s|,|\.|\:|\!|\?', text)
      word_counts = Counter(words)
      row['Text'] = word_counts
      yield row

with beam.Pipeline() as pipeline:
    news_df = pipeline | 'Read news csv' >> beam.dataframe.io.read_csv('bbc_news.csv')
    news_pcol = convert.to_pcollection(news_df) | beam.Map(lambda x: dict(x._asdict()))
    stopwords = (pipeline | 'Read stopwords' >> beam.io.ReadFromText('stopwords.txt'))
    ans_pcol = (news_pcol |
                "Split text" >> beam.ParDo(SplitText()) |
                "Filter words" >> beam.ParDo(FilterText(), beam.pvalue.AsList(stopwords)) |
                "Explode rows" >> beam.ParDo(ExplodeWords())
                )
    ans_df = convert.to_dataframe(ans_pcol | "To Rows" >> beam.Map(
        lambda x: beam.Row(
            ArticleId = x['ArticleId'],
            Category = x['Category'],
            Tag = x['Tag'],
            Frequency = x['Frequency'],
        )
    ))
    ans_df.to_csv("resultset.csv", index=False)  

/usr/local/lib/python3.7/dist-packages/apache_beam/dataframe/io.py:572: FutureWarning: WriteToFiles is experimental.
  sink=lambda _: _WriteToPandasFileSink(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/fileio.py:550: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  p.options.view_as(GoogleCloudOptions).temp_location or




---


შესრულებული დავალება მოგვაწოდეთ ნოუთბუქის სახით.
წარმატებები!